In [1]:
import sys
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from scipy.stats import qmc
import itertools
from itertools import combinations_with_replacement, combinations, permutations

import bo_methods_lib
from bo_methods_lib.bo_methods_lib.GPBO_Classes_New import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Class_fxns import * #Fix this later
from bo_methods_lib.bo_methods_lib.analyze_data import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Classes_plotters import * #Fix this later
import pympler
import pickle
import signac

from pympler import asizeof

from matplotlib import pyplot as plt

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 200

#Ignore inconcistent version warning
import warnings
from sklearn.exceptions import InconsistentVersionWarning
warnings.filterwarnings(action='ignore', category=InconsistentVersionWarning)

In [2]:
#From signac
import signac
meth_name_val_list = [1, 2, 3, 4, 5, 6]
save_csv = False

criteria_dict = {"cs_name_val" : 1,
                 "ep_enum_val": {"$gte": 1},
                 "meth_name_val": {"$in": meth_name_val_list}}

criteria_dict = {"cs_name_val" : 1,
                 "ep_enum_val": 1,
                 "meth_name_val": {"$in": meth_name_val_list}}

# criteria_dict = {"cs_name_val" : 1,
#                  "param_name_str" : "t1t2",
#                  "ep_enum_val":1,
#                  "retrain_GP": 25,
#                  "num_x_data": 5,
#                  "outputscl": None,
#                  "bo_iter_tot": 50,
#                  "lenscl": None}



In [7]:
project = signac.get_project()
save_csv = False
analyzer = Summarize_Data(criteria_dict, project, save_csv)

# df_all_jobs, job_list, theta_true = analyzer.get_df_all_jobs()
# print(min(df_all_jobs["Min Obj Act"]))
# print(df_all_jobs["BO Method"].unique())

df_best, job_list_best = analyzer.get_best_data()

In [8]:
df_best

,Run Number,BO Iter,Best Error,Exploration Bias,Max EI,Theta Max EI,Min Obj,Min Obj Act,Theta Min Obj,Min Obj Cum.,...,Regret,Speed,Time/Iter,EP Method Val,BO Method,Job ID,Max Evals,Termination,Total Run Time,L2 Norm Theta
408,3,15,5.342819,1.0,0.000000e+00,"[-0.3157894736842106, 1.3684210526315788]",2.306116,0.562206,"[0.8377381288793517, -0.9062360571368153]",0.140123,...,0.063249,0.016348,2.425027,CONSTANT,A1,7d335fac3f3dbef1def51beac296d8bf,16,ei-obj,38.800434,0.187405
787,1,9,-2.564069,1.0,0.000000e+00,"[0.3157894736842106, -0.5263157894736843]",-2.592402,-1.859226,"[0.9440185900221204, -1.0605013458550059]",-2.564066,...,0.009375,0.000768,2.883567,CONSTANT,B1,f77eb15458c0df1814cb2d097bc16619,27,ei-obj,77.856318,0.082428
670,2,20,0.000086,1.0,2.401248e-07,"[-0.10526315789473695, 0.10526315789473673]",0.000016,0.000264,"[0.9960445502815461, -0.9982226238924355]",0.00008,...,0.000227,0.003862,6.516778,CONSTANT,A2,bfaeb98a04d2a186bf9fc0c5a87602e7,26,obj,169.436222,0.004336
287,3,27,-10.519601,1.0,4.895559e+01,"[1.0004024855954028, -0.9992785636323397]",-12.183051,-10.852415,"[1.0004048758192912, -0.9992781038687927]",-10.852415,...,5.087350,0.285175,507.603594,CONSTANT,B2,6c8f446e38438959407a256119512c05,50,max_budget,25380.179691,0.000828
110,4,9,0.000086,1.0,1.381169e-06,"[1.096630551832445, -0.9917403943641504]",0.000011,0.000243,"[0.9951049264434847, -0.9996761634425393]",0.00008,...,0.000248,0.000177,8.957530,CONSTANT,C2,08978be5acb14bc5c527c23532eaa621,35,obj-regret,313.513559,0.004906
427,1,9,0.000076,1.0,2.504833e-07,"[1.096657295630544, -0.9917334311415682]",0.000007,0.000184,"[0.995800773902059, -0.9995407367818884]",0.000081,...,0.000240,0.000180,12.715346,CONSTANT,D2,811c3b5e401ba0adcedd67a67dbf2457,34,obj-regret,432.321759,0.004224


In [ ]:
### Get Best Data from ep experiment
df, job_list, theta_true = get_df_all_jobs(criteria_dict, save_csv)
print(min(df_all_jobs["Min Obj Act"]))
# # print(df.head())
# df_best, job_list_best = get_best_data(criteria_dict, df, job_list, theta_true, save_csv)

# for i in range(len(job_list_best)): 
#     print(job_list_best[i].id)
    
# df_best

In [ ]:
run_num_list = list(map(int, df_best["Run Number"].to_numpy()))
bo_iter_list = list(map(int, df_best["BO Iter"].to_numpy()))
bo_method_list = list(df_best["BO Method"].to_numpy())

In [ ]:
z_choices = ["min_sse", "sse"]

x_label = "BO Iterations"
y_label = r"$\mathbf{e(\theta)}$"
# y_label = "Max " + r"$\mathbf{EI(\theta)}$"
save_path = make_dir_name_from_criteria(criteria_dict)
plot_dict = make_plot_dict(False, None, x_label, None, line_levels = None, save_path=save_path)

file_path_list = [job.fn("BO_Results.gz") for job in job_list_best]

plot_objs_all_methods(file_path_list, run_num_list, z_choices, plot_dict)

In [ ]:
z_choice = "min_sse"

x_label = "BO Iterations"
# y_label = r"$\mathbf{e(\theta)}$"
y_label = "Max " + r"$\mathbf{EI(\theta)}$"
# title = "Min SSE Values"
save_path = make_dir_name_from_criteria(criteria_dict)
plot_dict = make_plot_dict(False, None, x_label, y_label, line_levels = None, save_path=save_path)

file_path_list = [job.fn("BO_Results.gz") for job in job_list]

plot_one_obj_all_methods(file_path_list, run_num_list, z_choice, plot_dict)

In [ ]:
pair = 0
z_choice = "sse_mean"
# title = "Heat Map Pair " + z_choice
levels = [100,100,100, 100, 100, 100]
save_path = make_dir_name_from_criteria(criteria_dict)
plot_dict = make_plot_dict(False, None, None, None, line_levels = levels, save_path=save_path)
file_path_list = [job.fn("BO_Results.gz") for job in job_list_best]

plot_hms_all_methods(file_path_list, run_num_list, bo_iter_list, pair, z_choice, plot_dict)

In [ ]:
#Make Heat maps for all pairs
z_choices = ["sse_sim", "sse_mean", "sse_var", "ei"]
levels = [100,100,100,100]

#Loop over best jobs
for i in range(len(job_list_best)):   
    #Set file path
    file_path = job_list_best[i].fn("BO_Results.gz")
    #Back out number of parameters
    string_val = df_best["Theta Min Obj"].iloc[0]
    try:
        numbers = [float(num) for num in string_val.replace('[', '').replace(']', '').split()]
    except:
        numbers = [float(num) for num in string_val]
        
    #Create list of parameter pair combinations
    dim_theta = len(np.array(numbers).reshape(-1, 1))
    dim_list = np.linspace(0, dim_theta-1, dim_theta)
    pairs = len((list(combinations(dim_list, 2))))
    
    #Loop over parameter pairs
    for pair in range(pairs):
        #Set save path
        save_path = make_dir_name_from_criteria(criteria_dict)
        plot_dict = make_plot_dict(False, None, None, None, line_levels = levels, save_path=save_path)
        plot_hms_gp_compare(file_path, run_num_list[i], bo_iter_list[i], pair, z_choices, plot_dict)

In [ ]:
#Get HP Plots
#Set parameters
file_path = "2023/09/05/Data_Files/CS2_BO_method_C2_ep_method_BOYLE.pickle"
file_path = "2023/09/05/Data_Files/CS2_BO_method_B1_ep_method_CONSTANT.pickle"
file_path = "workspace/811c3b5e401ba0adcedd67a67dbf2457/BO_Results.gz"

x_label = "BO Iterations"
y_label = "Value"
title = "Hyperparameter Values"

save_path = os.path.join(os.path.dirname(file_path), "line_plots", "hyperparams")
plot_dict = make_plot_dict(False, title, x_label, y_label, ybins = 7, line_levels = None, save_path=save_path)

hps, hp_names, hp_true = analyze_hypers(file_path, save_csv = True)
plot_2D_Data_w_BO_Iter(hps, hp_names, hp_true, plot_dict)

In [ ]:
#Get sse, log_sse, and ei plots
file_path = "workspace/811c3b5e401ba0adcedd67a67dbf2457/BO_Results.gz" #CS1 2D Best

z_choices = ["sse", "min_sse", "ei"]
z_choices_sort = sorted(z_choices, key=lambda x: ("sse", "min_sse", "ei").index(x))

x_label = "BO Iterations"
y_label = "Values"
title = None

save_path = os.path.join(os.path.dirname(file_path), "line_plots", '_'.join(map(str, z_choices_sort)))
plot_dict = make_plot_dict(False, title, x_label, y_label, ybins = 7, line_levels = None, save_path=save_path)

data, data_names, data_true, GP_meth_val = analyze_sse_min_sse_ei(file_path, z_choices)
plot_2D_Data_w_BO_Iter(data, data_names, data_true, plot_dict)

In [ ]:
#Get theta @ min sse
file_path = "2023/08/28/Data_Files/CS1_meth_A2_sep_fact_1.0.pickle"
file_path = "2023/09/05/Data_Files/CS2_BO_method_C2_ep_method_BOYLE.pickle"
file_path = "2023/09/05/Data_Files/CS2_BO_method_B1_ep_method_CONSTANT.pickle"
file_path = "workspace/811c3b5e401ba0adcedd67a67dbf2457/BO_Results.gz"

z_choice = "sse"

x_label = "BO Iterations"
title = "Min Obj Parameter Values"

save_path = os.path.join(os.path.dirname(file_path), "line_plots", z_choice + "_params")
plot_dict = make_plot_dict(False, title, x_label, None, line_levels = None, save_path=save_path)

data, data_names, data_true = analyze_thetas(file_path, z_choice)
plot_2D_Data_w_BO_Iter(data, data_names, data_true, plot_dict)

In [ ]:
#Get theta @ min sse overall
file_path = "workspace/811c3b5e401ba0adcedd67a67dbf2457/BO_Results.gz"

z_choice = "min_sse"

x_label = "BO Iterations"
title = "Min Obj Parameter Values Overall"

save_path = os.path.join(os.path.dirname(file_path), "line_plots", z_choice + "_params")
plot_dict = make_plot_dict(False, title, x_label, None, line_levels = None, save_path=save_path)

data, data_names, data_true = analyze_thetas(file_path, z_choice)
plot_2D_Data_w_BO_Iter(data, data_names, data_true, plot_dict)

In [ ]:
#Get theta @ Max EI
file_path = "2023/08/28/Data_Files/CS1_meth_A2_sep_fact_1.0.pickle"
file_path = "2023/09/05/Data_Files/CS2_BO_method_C2_ep_method_BOYLE.pickle"
file_path = "2023/09/05/Data_Files/CS2_BO_method_B1_ep_method_CONSTANT.pickle"
file_path = "workspace/811c3b5e401ba0adcedd67a67dbf2457/BO_Results.gz"

z_choice = "ei"

x_label = "BO Iterations"
y_label = "Theta Values"
title = "Max EI Parameter Values"

save_path = os.path.join(os.path.dirname(file_path), "line_plots", z_choice + "_params")
plot_dict = make_plot_dict(False, title, x_label, y_label, line_levels = None, save_path=save_path)

data, data_names, data_true = analyze_thetas(file_path, z_choice)
plot_2D_Data_w_BO_Iter(data, data_names, data_true, plot_dict)

In [ ]:
# #Get Median Data from sf experiment
df_median, job_list_med = get_median_data(criteria_dict, df, job_list, theta_true, save_csv)

for i in range(len(job_list_med)): 
    print(job_list_med[i].id)
    
df_median

In [ ]:
# #Get Mean Data from sf experiment
df_mean, job_list_mean = get_mean_data(criteria_dict, df, job_list, theta_true, save_csv)

for i in range(len(job_list_mean)): 
    print(job_list_mean[i].id)
    
df_mean